In [1]:
import os
import gc
import torch
import wandb
from PIL import Image
from transformers import AutoModel
from torchvision.transforms import v2
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torch.optim import AdamW, Adam, SGD
from torch import nn
from tqdm import tqdm

In [2]:
# Making datasets
def loader(path):
    img = Image.open(path)
    return img

transform = v2.Compose(
    [
        v2.Resize(224),
        v2.CenterCrop(224),
        v2.ToImage(), 
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ]
)
    
dataset = ImageFolder(
    'images/40X/', 
    loader=loader,
    transform=transform
)

# Split data into train/val/test
num_imgs = len(dataset.samples)
train_size = int(num_imgs * 0.7)
val_size = int(num_imgs * 0.15)
test_size = num_imgs - train_size - val_size

generator = torch.Generator().manual_seed(42)

train_dataset, val_dataset, test_dataset = random_split(
    dataset=dataset,
    lengths=[train_size, val_size, test_size],
    generator=generator
)

# Make dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Load Model and Create Classification Head

In [3]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

# Use GPU if avail
# Load model
with torch.no_grad():
    backbone = AutoModel.from_pretrained('kaiko-ai/midnight')

In [4]:
class PathBinaryClassifier(nn.Module):
    def __init__(self, backbone, hidden=512, dropout=0.2):
        super().__init__()

        self.backbone = backbone
        self.dim = backbone.config.hidden_size # 1536 for Midnight Model
        self.dropout = dropout

        # Classifier block
        self.classifier = nn.Sequential(
            nn.Linear(self.dim, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, 2), # 1 for BCELoss, 2 for CrossEntropyLoss
            # nn.Sigmoid() Needed for BCELoss, but not 
        )

    def forward(self, X):
        out = self.backbone(X)['pooler_output']
        logit = self.classifier(out)
        return logit

In [5]:
# Specify components needed for training

# NEED GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Instantiate model
model = PathBinaryClassifier(backbone).to(device)

In [6]:
# Freeze all layers except for those in classification head
for name, block in model.named_children():
    if name != 'classifier':
        for param in block.parameters():
            param.requires_grad = False

In [7]:
# Specify opt and loss function
optimizer = AdamW(filter(lambda p: p.requires_grad == True, model.parameters()), lr=2e-5)
criterion = nn.CrossEntropyLoss()

In [8]:
# Specify Wandb config
project = 'PathologyFineTuning'
config = {
    'architecture': 'Midnight-12k',
    'dataset': 'BreakHis',
    'optimizer': 'AdamW',
    'classifier_hidden_dim': 512,
    'epochs': 2,
}
name = 'trial-run'

# Define config for Sweep
sweep_config = {
    'method' : 'random',
    'metric' : {'name': 'val_acc', 'goal': 'maximize'},
    'parameters' : {
        'classifier_hidden_dim' : {'min': 32, 'max': 2048},
        'init_lr' : {'min': 1e-7, 'max': 1e-3},
        'dropout' : {'min': 0.1, 'max': 0.5},
        'optimizer' : {'values' : ['AdamW', 'Adam', 'SGD']}
    }
}

In [9]:
# Function for hyperparameter sweep using Wandb
def sweep_train():
    with wandb.init() as run:
        config = wandb.config
        hidden_dim = config.classifier_hidden_dim
        opt_choice = config.optimizer
        init_lr = config.init_lr
        dropout = config.dropout

        # Define model based on HP choice
        model = PathBinaryClassifier(
            backbone=backbone, 
            hidden=hidden_dim, 
            dropout=dropout
        ).to(device)

        # Setup optimizer based on HP choice
        if opt_choice == 'AdamW':
            opt = AdamW(model.parameters(), lr=init_lr)
        elif opt_choice == 'Adam':
            opt = Adam(model.parameters(), lr=init_lr)
        elif opt_choice == 'SGD':
            opt = SGD(model.parameters(), lr=init_lr)

        # Train model with HPs choice by sweep
        train(
            model,
            train_dataloader,
            val_dataloader,
            opt,
            criterion
        )

In [10]:
def train(model, train_dataloader, val_dataloader, opt, l, epochs=3, grad_accum=4):
    # Use wandb for tracking
    # with wandb.init(project=project, config=config, name=name) as run:
    run = wandb.run # if initialized sweep
    
    epoch_train_loss = []
    epoch_train_acc = []
    epoch_val_acc = []
    best_val_acc = 0
    
    for i in range(epochs):
        running_loss = 0
        running_correct_train = 0
        running_correct_val = 0
        
        print(f'---- Epoch {i+1}/{epochs}----')
        opt.zero_grad()

        # training
        for X, y in tqdm(train_dataloader):
            X, y = X.to(device), y.to(device)

            # Get predictions
            output = model(X)

            # Compute loss and gradients
            loss = l(output, y)
            running_loss += loss
            loss.backward()

            # Count correct predictions
            preds = torch.argmax(output, dim=1)
            running_correct_train += sum(preds == y).item()
            
            # Update parameters after {grad_accum} batches
            if (i+1) % grad_accum == 0:
                opt.step()
                opt.zero_grad()

            opt.step()
            opt.zero_grad()

        # Metrics for training epoch
        cur_train_loss = running_loss / len(train_dataloader.dataset)
        cur_train_acc = running_correct_train / len(train_dataloader.dataset)
        epoch_train_loss.append(cur_train_loss)
        epoch_train_acc.append(cur_train_acc)

        print(f'---- Epoch {i+1}/{epochs} Train Loss: {cur_train_loss} --- Train Accuracy: {cur_train_acc} ----')

        # Wandb logging
        run.log({'train_loss': cur_train_loss, 'train_acc': cur_train_acc})
        
        # validation
        for X, y in tqdm(val_dataloader):
            X, y, = X.to(device), y.to(device)

            # Get predictions
            output = model(X)
            
            # Count correct predictions
            preds = torch.argmax(output, dim=1)
            
            running_correct_val += sum(preds == y).item()


        # Metrics for validation epoch
        cur_val_acc = running_correct_val / len(val_dataloader.dataset)
        epoch_val_acc.append(cur_val_acc)
        
        if cur_val_acc > best_val_acc:
            best_val_acc = cur_val_acc            
                                                
        print(f'---- Epoch {i+1}/{epochs} Val Accuracy: {cur_val_acc} ----')

        # Wandb logging
        run.log({'val_acc': cur_val_acc})
    
    print('Best val acc: ', best_val_acc)
    
    return model, best_val_acc

In [11]:
# Create sweep
sweep_id = wandb.sweep(sweep_config, project=project)

Create sweep with ID: 7qtpgyud
Sweep URL: https://wandb.ai/team-chucklemunch/PathologyFineTuning/sweeps/7qtpgyud


In [12]:
# Launch sweep
wandb.agent(sweep_id, function=sweep_train, count=5)

wandb: Agent Starting Run: brv6ksy1 with config:
wandb: 	classifier_hidden_dim: 1588
wandb: 	dropout: 0.3854153744000904
wandb: 	init_lr: 0.0003119270217988778
wandb: 	optimizer: Adam
wandb: Currently logged in as: charlie-kotula (team-chucklemunch) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


---- Epoch 1/3----


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:45<00:00,  5.13s/it]


---- Epoch 1/3 Train Loss: 0.009198344312608242 --- Train Accuracy: 0.8739255014326648 ----


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:49<00:00,  4.93s/it]


...

---- Epoch 3/3 Val Accuracy: 0.9698996655518395 ----
Best val acc:  0.9732441471571907


train_acc,▁▇█
train_loss,█▂▁
val_acc,▁█▇
train_acc,0.97135
train_loss,0.00282
val_acc,0.9699


wandb: Agent Starting Run: dzpaqaal with config:
wandb: 	classifier_hidden_dim: 108
wandb: 	dropout: 0.13411730994699128
wandb: 	init_lr: 0.0009727065256760924
wandb: 	optimizer: AdamW


---- Epoch 1/3----


 45%|██████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                                 | 20/44 [01:43<02:03,  5.16s/it]wandb: Ctrl + C detected. Stopping sweep.


In [13]:
# # Training without doing sweep
# model, best_val_acc = train(model, 
#                             train_dataloader, 
#                             val_dataloader, 
#                             optimizer, 
#                             criterion)